In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
# define data directory

DATA_DIR = Path('../data/raw')
files = {
    'circuits': DATA_DIR /'circuits.csv',
    'constructor_results': DATA_DIR /'constructor_results.csv',
    'constructor_standings': DATA_DIR /'constructor_standings.csv',
    'constructors': DATA_DIR /'constructors.csv',
    'driver_standings': DATA_DIR /'driver_standings.csv',
    'drivers': DATA_DIR /'drivers.csv',
    'lap_times': DATA_DIR /'lap_times.csv',
    'pit_stops': DATA_DIR /'pit_stops.csv',
    'qualifying': DATA_DIR /'qualifying.csv',
    'races': DATA_DIR /'races.csv',
    'results': DATA_DIR /'results.csv',
    'seasons': DATA_DIR /'seasons.csv',
    'sprint_results': DATA_DIR /'sprint_results.csv',
    'status': DATA_DIR /'status.csv',
}

In [5]:
print("Loading all CSV files...\n")

circuits = pd.read_csv(files['circuits'])
constructor_results = pd.read_csv(files['constructor_results'])
constructor_standings = pd.read_csv(files['constructor_standings'])
constructors = pd.read_csv(files['constructors'])
driver_standings = pd.read_csv(files['driver_standings'])
drivers = pd.read_csv(files['drivers'])
lap_times = pd.read_csv(files['lap_times'])
pit_stops = pd.read_csv(files['pit_stops'])
qualifying = pd.read_csv(files['qualifying'])
races = pd.read_csv(files['races'])
results = pd.read_csv(files['results'])
seasons = pd.read_csv(files['seasons'])
sprint_results = pd.read_csv(files['sprint_results'])
status = pd.read_csv(files['status'])

print(f" circuits: {len(circuits)} rows")
print(f" constructor_results: {len(constructor_results)} rows")
print(f" constructor_standings: {len(constructor_standings)} rows")
print(f" constructors: {len(constructors)} rows")
print(f" driver_standings: {len(driver_standings)} rows")
print(f" drivers: {len(drivers)} rows")
print(f" lap_times: {len(lap_times)} rows")
print(f" pit_stops: {len(pit_stops)} rows")
print(f" qualifying: {len(qualifying)} rows")
print(f" races: {len(races)} rows")
print(f" results: {len(results)} rows")
print(f" seasons: {len(seasons)} rows")
print(f" sprint_results: {len(sprint_results)} rows")
print(f" status: {len(status)} rows")

Loading all CSV files...

 circuits: 77 rows
 constructor_results: 12625 rows
 constructor_standings: 13391 rows
 constructors: 212 rows
 driver_standings: 34863 rows
 drivers: 861 rows
 lap_times: 589081 rows
 pit_stops: 11371 rows
 qualifying: 10494 rows
 races: 1125 rows
 results: 26759 rows
 seasons: 75 rows
 sprint_results: 360 rows
 status: 139 rows


In [ ]:
# Merge the files

df = results.copy()
print(f"Starting with results: {df.shape}")

# add race information
df = df.merge(
    races[['raceId', 'year', 'round', 'circuitId', 'name', 'date']], 
    on='raceId', 
    how='left'
)
print(f"After adding race info: {df.shape}")
display(df.head(3))

# add driver information
df = df.merge(
    drivers[['driverId', 'driverRef', 'code', 'forename', 'surname', 'nationality']], 
    on='driverId', 
    how='left'
)
print(f"After adding driver info: {df.shape}")
display(df.head(3))

# add constructor information
df = df.merge(
    constructors[['constructorId', 'constructorRef', 'name', 'nationality']], 
    on='constructorId', 
    how='left',
    suffixes=('_race', '_constructor')
)
print(f"After adding constructor info: {df.shape}")
display(df.head(3))

# add qualifying data
df = df.merge(
    qualifying[['raceId', 'driverId', 'position']], 
    on=['raceId', 'driverId'], 
    how='left',
    suffixes=('', '_qualifying')
)
print(f"After adding qualifying: {df.shape}")
display(df.head(3))

# add circuit information
df = df.merge(
    circuits[['circuitId', 'circuitRef', 'name', 'location', 'country']], 
    on='circuitId', 
    how='left',
    suffixes=('_race', '_circuit')
)
print(f"Final merged shape: {df.shape}")
display(df.head(3))

# clean the data

# Remove rows without finish position
df = df[df['positionOrder'].notna()]

# Convert to numeric
df['grid'] = pd.to_numeric(df['grid'], errors='coerce')
df['position_qualifying'] = pd.to_numeric(df['position_qualifying'], errors='coerce')

# Rename columns for clarity
df = df.rename(columns={
    'positionOrder': 'finish_position',
    'position_qualifying': 'grid_position',
    'name_race': 'race_name',
    'name_circuit': 'circuit_name',
    'name_constructor': 'constructor_name'
})

print(f"Cleaned data shape: {df.shape}")

# data quality checks

print("DATA QUALITY REPORT")

print(f"\n Basic Stats:")
print(f"  - Total records: {len(df):,}")
print(f"  - Years: {df['year'].min()} to {df['year'].max()}")
print(f"  - Unique races: {df['raceId'].nunique()}")
print(f"  - Unique drivers: {df['driverId'].nunique()}")
print(f"  - Unique constructors: {df['constructorId'].nunique()}")

print(f"\n Missing Values:")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    display(missing)
else:
    print("No missing values!")

print(f"\n Target Variable (finish_position):")
print(f"  - Min: {df['finish_position'].min()}")
print(f"  - Max: {df['finish_position'].max()}")
print(f"  - Mean: {df['finish_position'].mean():.2f}")

# save into data/processed
output_path = Path('../data/processed/merged_data.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)

Starting with results: (26759, 18)
After adding race info: (26759, 23)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name,date
0,1,18,1,1,22,1,1,1,1,10.0,...,39,2,1:27.452,218.300,1,2008,1,1,Australian Grand Prix,2008-03-16
1,2,18,2,2,3,5,2,2,2,8.0,...,41,3,1:27.739,217.586,1,2008,1,1,Australian Grand Prix,2008-03-16
2,3,18,3,3,7,7,3,3,3,6.0,...,41,5,1:28.090,216.719,1,2008,1,1,Australian Grand Prix,2008-03-16


After adding driver info: (26759, 28)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,year,round,circuitId,name,date,driverRef,code,forename,surname,nationality
0,1,18,1,1,22,1,1,1,1,10.0,...,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,Lewis,Hamilton,British
1,2,18,2,2,3,5,2,2,2,8.0,...,2008,1,1,Australian Grand Prix,2008-03-16,heidfeld,HEI,Nick,Heidfeld,German
2,3,18,3,3,7,7,3,3,3,6.0,...,2008,1,1,Australian Grand Prix,2008-03-16,rosberg,ROS,Nico,Rosberg,German


After adding constructor info: (26759, 31)


...

Final merged shape: (26759, 36)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,surname,nationality_race,constructorRef,name_constructor,nationality_constructor,position_qualifying,circuitRef,name,location,country
0,1,18,1,1,22,1,1,1,1,10.0,...,Hamilton,British,mclaren,McLaren,British,1.0,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia
1,2,18,2,2,3,5,2,2,2,8.0,...,Heidfeld,German,bmw_sauber,BMW Sauber,German,5.0,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia
2,3,18,3,3,7,7,3,3,3,6.0,...,Rosberg,German,williams,Williams,British,7.0,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia


Cleaned data shape: (26759, 36)
DATA QUALITY REPORT

 Basic Stats:
  - Total records: 26,759
  - Years: 1950 to 2024
  - Unique races: 1125
  - Unique drivers: 861
  - Unique constructors: 211

 Missing Values:


grid_position    16265
dtype: int64


 Target Variable (finish_position):
  - Min: 1
  - Max: 39
  - Mean: 12.79
